# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,puerto ayora,-0.7393,-90.3518,24.19,95,18,1.79,EC,1691131656
1,1,kani keli,-12.9536,45.1033,28.59,78,31,8.50,YT,1691131657
2,2,port-aux-francais,-49.3500,70.2167,2.58,88,70,14.48,TF,1691131657
3,3,vorgashor,67.5833,63.9500,27.60,44,0,2.26,RU,1691131657
4,4,jamestown,42.0970,-79.2353,21.77,94,0,1.54,US,1691131618


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display
# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng", 
    "Lat", 
    geo=True,
    tiles="OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 0.5,
    color = "City"
)
# Display the map plot
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather_cities = city_data_df[(city_data_df['Max Temp'] > 15) & (city_data_df['Max Temp'] < 28)& \
                                    (city_data_df['Cloudiness'] >20) & (city_data_df['Humidity'] < 50)]

# Drop any rows with null values
ideal_weather_cities = ideal_weather_cities.dropna(how='any')

# Display sample data
ideal_weather_cities

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
22,22,bukama,-9.2000,25.8500,25.59,34,39,2.25,CD,1691131662
28,28,port alfred,-33.5906,26.8910,21.77,37,48,3.13,ZA,1691131664
51,51,ust-nera,64.5667,143.2000,20.91,27,37,2.86,RU,1691131670
62,62,laascaanood,8.4774,47.3597,27.72,42,29,16.06,SO,1691131674
79,79,talnakh,69.4865,88.3972,22.34,33,28,3.29,RU,1691131678
81,81,chadan,51.2833,91.5833,27.85,33,65,1.02,RU,1691131678
133,133,verkhnevilyuysk,63.4453,120.3167,27.03,28,100,3.50,RU,1691131695
144,144,sao vicente,-23.9631,-46.3919,20.55,26,21,1.94,BR,1691131646
168,168,susuman,62.7833,148.1667,16.99,33,69,7.95,RU,1691131704
179,179,jiangzi,28.9500,89.6333,15.76,39,100,2.23,CN,1691131710


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_cities.loc[:, ['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
22,bukama,CD,-9.2000,25.8500,34,
28,port alfred,ZA,-33.5906,26.8910,37,
51,ust-nera,RU,64.5667,143.2000,27,
62,laascaanood,SO,8.4774,47.3597,42,
79,talnakh,RU,69.4865,88.3972,33,
81,chadan,RU,51.2833,91.5833,33,
133,verkhnevilyuysk,RU,63.4453,120.3167,28,
144,sao vicente,BR,-23.9631,-46.3919,26,
168,susuman,RU,62.7833,148.1667,33,
179,jiangzi,CN,28.9500,89.6333,39,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 2000
params = {
    "categories":"accommodation.hotel",
    "apiKey":geoapify_key,
    "format":"json"
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    response = requests.get(base_url,params=params)
    # Convert the API response to JSON format
    name_address = response.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
bukama - nearest hotel: Mission catholique
port alfred - nearest hotel: No hotel found
ust-nera - nearest hotel: гостиница "Солнечная"
laascaanood - nearest hotel: Warsame Hotel mnesta
talnakh - nearest hotel: Талнах
chadan - nearest hotel: Жарки
verkhnevilyuysk - nearest hotel: No hotel found
sao vicente - nearest hotel: No hotel found
susuman - nearest hotel: No hotel found
jiangzi - nearest hotel: No hotel found
okhotsk - nearest hotel: No hotel found
bodaybo - nearest hotel: НОЙ
pyt-yakh - nearest hotel: Сибирь
usogorsk - nearest hotel: No hotel found
arenapolis - nearest hotel: No hotel found
broken hill - nearest hotel: No hotel found
mgandu - nearest hotel: No hotel found
plettenberg bay - nearest hotel: Milkwood Manor
vikhorevka - nearest hotel: Сосновые родники
uliastay - nearest hotel: Болоржин зочид буудал
kruisfontein - nearest hotel: No hotel found
taksimo - nearest hotel: No hotel found
altai - nearest hotel: Altai hotel 40000-100000T
lensk - nearest

,City,Country,Lat,Lng,Humidity,Hotel Name
22,bukama,CD,-9.2000,25.8500,34,Mission catholique
28,port alfred,ZA,-33.5906,26.8910,37,No hotel found
51,ust-nera,RU,64.5667,143.2000,27,"гостиница ""Солнечная"""
62,laascaanood,SO,8.4774,47.3597,42,Warsame Hotel mnesta
79,talnakh,RU,69.4865,88.3972,33,Талнах
81,chadan,RU,51.2833,91.5833,33,Жарки
133,verkhnevilyuysk,RU,63.4453,120.3167,28,No hotel found
144,sao vicente,BR,-23.9631,-46.3919,26,No hotel found
168,susuman,RU,62.7833,148.1667,33,No hotel found
179,jiangzi,CN,28.9500,89.6333,39,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.loc[hotel_df["Hotel Name"]!="No hotel found"].hvplot.points(
    "Lng", 
    "Lat", 
    geo=True,
    tiles="OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 0.5,
    color = "City",
    hover_cols=["Hotel Name","Country"]
)
# Display the map plot
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)